In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json

In [4]:
##FFmpg (turns your gource into a MP4)
#brew install ffmpeg

In [5]:
##when that is finished installing type 
#brew install gource

**Run this with**:
- gource custom.log --seconds-per-day 1 --bloom-intensity 0.3
- gource custom.log --seconds-per-day 1 --bloom-intensity 0.3 -e 0.2 -i 120
- gource custom.log --seconds-per-day 1 --bloom-intensity 0.3 -e 0.2 -i 120 --title "Development of the amount of characters in Rick and Morty"
- gource custom.log --seconds-per-day 1 -1440x900 --bloom-intensity 0.1 -e 0.2 -i 120 --title "Development of the amount of characters in Rick and Morty"
- gource custom.log -1280x720 --seconds-per-day 2 --key --hide filenames

**Output video**:
- gource custom.log -1280x720 --seconds-per-day 2 --key --hide dirnames, -o - | ffmpeg -y -r 60 -f image2pipe -vcodec ppm -i - -vcodec libx264 -preset ultrafast -crf 1 -threads 0 -bf 0 output.mp4

- link : [custom log format](https://github.com/acaudwell/Gource/wiki/Custom-Log-Format)
- link : [visualization](https://edgeryders.eu/t/visualizing-our-7-years-of-online-dialogue-with-gource/11905)
- likn : [stuff to gource](https://leonjza.github.io/blog/2013/07/27/stuff-to-gource./#get-the-code-setup-your-own)

The software to create this is Gource 2, an open source tool originally meant to visualize Git repositories. Fortunately it also can read a custom log format 9.

For Discourse, the custom log file would be a text file with each line containing the following components, delimited by `|`:

- timestamp
- username
- type of action (Added, Modified, Deleted); the “M” is useful esp. for wikis
- path to the affected resource. Usually this applies to files, but in our case it would be about posts. This is the part we’d have to adapt to Discourse. Our path format could be `category/subcategory/topic-id/post-id` or it could even include the full threading information with any depth of nesting, such as `category/subcategory/topic-id/parent-post-id/parent-post-id/post-id`
- color for the affected resource (optional); this could be chosen according to the category color on edgeryders.eu

Example line:

- `1275543595|matthias|A|earthos/reef/1234/1`


In [2]:
import pandas as pd
import pandasql as ps
import numpy as np
import re
import tqdm
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [3]:
df_characters = pd.read_csv('../df_characters.csv')
df_episodes = pd.read_csv('../df_episodes.csv')
df_transcripts = pd.read_csv('../df_transcripts.csv')
df_rel_sent = pd.read_csv('../df_rel_sent.csv')

In [4]:
from ast import literal_eval
df_characters['episode'] = df_characters['episode'].apply(literal_eval)

In [5]:
df_characters_exploded = df_characters.explode('episode')

In [6]:
df_characters_exploded.episode = df_characters_exploded.episode.astype(int)

In [7]:
df_characters_exploded

,name,status,species,type,gender,origin,location,image,episode,url,created,references
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,1,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,2,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,3,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,4,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,5,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
821,Young Jerry,unknown,Human,NaN,Male,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,51,https://rickandmortyapi.com/api/character/822,2021-11-02T17:18:31.934Z,NaN
822,Young Beth,unknown,Human,NaN,Female,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,51,https://rickandmortyapi.com/api/character/823,2021-11-02T17:19:00.951Z,NaN
823,Young Beth,unknown,Human,NaN,Female,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,51,https://rickandmortyapi.com/api/character/824,2021-11-02T17:19:47.957Z,NaN
824,Young Jerry,unknown,Human,NaN,Male,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,51,https://rickandmortyapi.com/api/character/825,2021-11-02T17:20:14.305Z,NaN


In [8]:
import datetime
from datetime import datetime
from datetime import timedelta
import time

In [215]:
date_range = pd.date_range(datetime.today() - timedelta(days=51), periods=51)
date_series = pd.Series(date_range)
df_episode_unix_timestamp = pd.DataFrame(pd.DataFrame(date_series).values[::-1])\
    .reset_index(drop=False)\
    .rename(columns={'index':'episode', 0:'timestamp'})
df_episode_unix_timestamp['episode'] = df_episode_unix_timestamp['episode'].apply(lambda x: x+1)
df_episode_unix_timestamp['unixtime'] = df_episode_unix_timestamp['timestamp'].astype(int)
df_episode_unix_timestamp


,episode,timestamp,unixtime
0,1,2021-11-30 12:03:08.828821,1638273788828821000
1,2,2021-11-29 12:03:08.828821,1638187388828821000
2,3,2021-11-28 12:03:08.828821,1638100988828821000
3,4,2021-11-27 12:03:08.828821,1638014588828821000
4,5,2021-11-26 12:03:08.828821,1637928188828821000
5,6,2021-11-25 12:03:08.828821,1637841788828821000
6,7,2021-11-24 12:03:08.828821,1637755388828821000
7,8,2021-11-23 12:03:08.828821,1637668988828821000
8,9,2021-11-22 12:03:08.828821,1637582588828821000
9,10,2021-11-21 12:03:08.828821,1637496188828821000


In [228]:
date_range = pd.date_range(datetime.today() - timedelta(days=19), periods=19)
date_series = pd.Series(date_range)
df_episode_unix_timestamp = pd.DataFrame(pd.DataFrame(date_series))\
    .reset_index(drop=False)\
    .rename(columns={'index':'episode', 0:'timestamp'})
df_episode_unix_timestamp['episode'] = df_episode_unix_timestamp['episode'].apply(lambda x: x+1)
df_episode_unix_timestamp['unixtime'] = df_episode_unix_timestamp['timestamp'].astype(int)
df_episode_unix_timestamp['unixtime'] = (df_episode_unix_timestamp.timestamp - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')


In [229]:
df_episode_unix_timestamp

,episode,timestamp,unixtime
0,1,2021-11-12 12:12:15.702384,1636719135
1,2,2021-11-13 12:12:15.702384,1636805535
2,3,2021-11-14 12:12:15.702384,1636891935
3,4,2021-11-15 12:12:15.702384,1636978335
4,5,2021-11-16 12:12:15.702384,1637064735
5,6,2021-11-17 12:12:15.702384,1637151135
6,7,2021-11-18 12:12:15.702384,1637237535
7,8,2021-11-19 12:12:15.702384,1637323935
8,9,2021-11-20 12:12:15.702384,1637410335
9,10,2021-11-21 12:12:15.702384,1637496735


The software to create this is Gource 2, an open source tool originally meant to visualize Git repositories. Fortunately it also can read a custom log format 9.

For Discourse, the custom log file would be a text file with each line containing the following components, delimited by `|`:

- timestamp
- username
- type of action (Added, Modified, Deleted); the “M” is useful esp. for wikis
- path to the affected resource. Usually this applies to files, but in our case it would be about posts. This is the part we’d have to adapt to Discourse. Our path format could be `category/subcategory/topic-id/post-id` or it could even include the full threading information with any depth of nesting, such as `category/subcategory/topic-id/parent-post-id/parent-post-id/post-id`
- color for the affected resource (optional); this could be chosen according to the category color on edgeryders.eu

Example line:

- `1275543595|matthias|A|earthos/reef/1234/1`


In [230]:
#df_characters_exploded

# Character pr. episode (custom_char_episode.log)

- 1633360391|episode_1|A|episode_1/rick/rick.cpp
- 1633360391|episode_1|A|episode_1/morty/rick.cpp
- 1633360391|episode_1|A|episode_1/summer/rick.cpp
- 1633360391|episode_2|A|episode_2/rick.cpp
- 1633360391|episode_3|A|episode_3/rick.cpp

In [14]:
res = ps.sqldf(
    """
    select 
        1633360391 || '|' || 'episode_' || episode || '|' || 'A' || '|' || 'episode_' || episode || '/' || character || '/' || 'player.cpp' as output
    from
    
    (
    
    select 
       episode
    ,  character
    from 
        (

        select
              replace(df.name, ' ', '_') as character
            , df.episode
        from df_characters_exploded df
        order by df.episode asc
        )
    order by episode asc
    )
    """
)

res

,output
0,1633360391|episode_1|A|episode_1/Rick_Sanchez/...
1,1633360391|episode_1|A|episode_1/Morty_Smith/p...
2,1633360391|episode_1|A|episode_1/Bepisian/play...
3,1633360391|episode_1|A|episode_1/Beth_Smith/pl...
4,1633360391|episode_1|A|episode_1/Canklanker_Th...
...,...
1262,1633360391|episode_51|A|episode_51/Redhead_Mor...
1263,1633360391|episode_51|A|episode_51/Young_Jerry...
1264,1633360391|episode_51|A|episode_51/Young_Beth/...
1265,1633360391|episode_51|A|episode_51/Young_Beth/...


In [15]:
with open('custom_char_episode.log', 'w') as f:
    for row in res.output:
        #print(row)
        f.write(row + '\n')

# Episode, Sentiment pr Character (top 5 talking pr episode)

The idea about this gource is to:
- Episodes are branches to eachother (linked)
- For each episode, take the top 5 (total sentences) characters
- Color the character according to sentiment

In [226]:
#df_rel_sent

In [227]:
#df_episodes

In [240]:
temp_df = ps.sqldf(
    """
    
    select 
        episode
      , alias as character
      , compound as sentiment
      , rn
    from
    (
    select 
      df.*
    , row_number() over (partition by episode order by num_sentences desc) as rn
    from df_rel_sent as df
    )
    
    where rn <= 5
    """
)

- `1275543595|matthias|A|earthos/reef/1234/1`

In [241]:
temp_df.head(10)

,episode,character,sentiment,rn
0,Anatomy Park,Rick,0.8118,1
1,Anatomy Park,Morty,-0.5759,2
2,Anatomy Park,Jerry,0.9647,3
3,Anatomy Park,Annie,0.8002,4
4,Anatomy Park,Dr. Bloom,0.0000,5
5,Auto Erotic Assimilation,Rick,0.5279,1
6,Gotron Jerrysis Rickvangelion,Rick,0.9944,1
7,Gotron Jerrysis Rickvangelion,Summer,0.9942,2
8,Gotron Jerrysis Rickvangelion,Morty,0.6733,3
9,Gotron Jerrysis Rickvangelion,Beth,0.8852,4


In [242]:
def rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))
    r = int(max(0, 255*(ratio - 1)))
    g = 0#255 - b - r
    return r, g, b


def rgb_to_hex(rgb):
    return '%02x%02x%02x' % rgb

In [243]:
rgb_to_hex(rgb(1, -1, -1))

'ff0000'

In [244]:
# As we have 19 episodes
df_episode_unix_timestamp

,episode,timestamp,unixtime
0,1,2021-11-12 12:12:15.702384,1636719135
1,2,2021-11-13 12:12:15.702384,1636805535
2,3,2021-11-14 12:12:15.702384,1636891935
3,4,2021-11-15 12:12:15.702384,1636978335
4,5,2021-11-16 12:12:15.702384,1637064735
5,6,2021-11-17 12:12:15.702384,1637151135
6,7,2021-11-18 12:12:15.702384,1637237535
7,8,2021-11-19 12:12:15.702384,1637323935
8,9,2021-11-20 12:12:15.702384,1637410335
9,10,2021-11-21 12:12:15.702384,1637496735


In [249]:
res = pd.merge(
    temp_df,
    df_episodes[['name', 'id']],
    how='left',
    left_on='episode',
    right_on='name'
).sort_values(['id', 'rn'], ascending=True).reset_index(drop=True)
res

,episode,character,sentiment,rn,name,id
0,Pilot,Rick,0.9975,1,Pilot,1
1,Pilot,Morty,0.9971,2,Pilot,1
2,Pilot,Jerry,0.9619,3,Pilot,1
3,Pilot,Beth,0.8590,4,Pilot,1
4,Pilot,Mr. Goldenfold,0.6080,5,Pilot,1
...,...,...,...,...,...,...
106,Rickmurai Jack,Rick,0.9806,1,Rickmurai Jack,51
107,Rickmurai Jack,Morty,0.9892,2,Rickmurai Jack,51
108,Rickmurai Jack,President Morty,0.8430,3,Rickmurai Jack,51
109,Rickmurai Jack,Crowscare,0.9445,4,Rickmurai Jack,51


In [271]:
res[res.episode =='Anatomy Park']

,episode,character,sentiment,rn,name,id
10,Anatomy Park,Rick,0.8118,1,Anatomy Park,3
11,Anatomy Park,Morty,-0.5759,2,Anatomy Park,3
12,Anatomy Park,Jerry,0.9647,3,Anatomy Park,3
13,Anatomy Park,Annie,0.8002,4,Anatomy Park,3
14,Anatomy Park,Dr. Bloom,0.0000,5,Anatomy Park,3


In [290]:
with open('episode_character_sentiment.log', 'w+') as f:
        

    n = 0
    for idx, row in res.iterrows():
        #print('1633360391' + '|' + str(row.episode))
        if idx == 0:
            episode = res.iloc[idx].episode
            episode_gathered = res.iloc[idx].episode
            folder = episode_gathered + '/'
            #print('idx0 :', episode)
            #print(res[res.episode == episode].character)
            #print(timestamp + '|' + 'sentiment|A|' + folder)
            f.write(timestamp + '|' + 'sentiment|A|' + folder + '\n')
            timestamp = str(df_episode_unix_timestamp.iloc[n].unixtime)
            for char in res[res.episode == episode].character:
                #print(episode + '/' + char + '.txt')
                #print(res[(res.episode == episode) & (res.character == char)].sentiment.values[0])
                sentiment_score = res[(res.episode == episode) & (res.character == char)].sentiment.values[0]
                sentiment_to_hex = rgb_to_hex(rgb(1, -1, sentiment_score))
                #print(sentiment_to_hex)
                #print(episode + '/' + char + '.txt' + '|' + sentiment_to_hex.upper())
                add_file = episode_gathered + '/' + char + '|' + sentiment_to_hex.upper()
                print(timestamp + '|' + 'sentiment|A|' + add_file)
                f.write(timestamp + '|' + 'sentiment|A|' + add_file + '\n')
                
        if idx != 0:
            episode = res.iloc[idx].episode 
            #for char in res[res.episode == episode].character:
            #    print(char)
            if episode != res.iloc[idx - 1].episode:
                episode = res.iloc[idx].episode
                if n == 0:
                    n += 1
                    continue
                try:
                    timestamp = str(df_episode_unix_timestamp.iloc[n].unixtime)
                except:
                    None
                #print('idx :', episode, res.iloc[idx].episode)
                episode_gathered += '/' + res.iloc[idx].episode
                if n % 3 == 0:
                    episode_gathered = res.iloc[idx].episode
                folder = episode_gathered + '/'
                print(timestamp + '|' + 'sentiment|A|' + folder)
                f.write(timestamp + '|' + 'sentiment|A|' + folder + '\n')
                #print('gathered : ', episode_gathered)
                for char in res[res.episode == episode].character:
                    sentiment_score = res[(res.episode == episode) & (res.character == char)].sentiment.values[0]
                    sentiment_to_hex = rgb_to_hex(rgb(1, -1, sentiment_score))
                    add_file = episode_gathered + '/' + char + '|' + sentiment_to_hex.upper()
                    print(timestamp + '|' + 'sentiment|A|' + add_file)
                    f.write(timestamp + '|' + 'sentiment|A|' + add_file + '\n')
                    
                n += 1
                #print(folder + '\n')
                #print('now : ', res.iloc[idx].episode)
                #print('before : ', res.iloc[idx-1].episode)
                #print(episode)
                #break
                

1636719135|sentiment|A|Pilot/Rick|0000FE
1636719135|sentiment|A|Pilot/Morty|0000FE
1636719135|sentiment|A|Pilot/Jerry|0000F5
1636719135|sentiment|A|Pilot/Beth|0000DB
1636719135|sentiment|A|Pilot/Mr. Goldenfold|00009B
1636805535|sentiment|A|Pilot/Anatomy Park/
1636805535|sentiment|A|Pilot/Anatomy Park/Rick|0000CF
1636805535|sentiment|A|Pilot/Anatomy Park/Morty|920000
1636805535|sentiment|A|Pilot/Anatomy Park/Jerry|0000F5
1636805535|sentiment|A|Pilot/Anatomy Park/Annie|0000CC
1636805535|sentiment|A|Pilot/Anatomy Park/Dr. Bloom|000000
1636891935|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/
1636891935|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/Rick|0000FE
1636891935|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/Morty|0000E8
1636891935|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/Jerry|0000FB
1636891935|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/Prince Nebulon|0000FC
1636891935|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/Mr. Marklev

**Note:**
- Run with:
    - `gource episode_character_sentiment.log --seconds-per-day 1 --bloom-intensity 0.3 -e 0.2 -i 120 --title "Sentiment for the top 5 Characters pr Episode"`
    - `gource episode_character_sentiment.log --seconds-per-day 2 --bloom-intensity 0.3 -1440x900 --title "Sentiment for the Main Characters pr Episode"`
    - `--background-image`
    - `/Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/prova8.png`
    
    - `gource episode_character_sentiment.log --seconds-per-day 2 --bloom-intensity 0.3 -1440x900 --title "Sentiment for the Main Characters pr Episode" --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/prova8.png`
    
    - `gource episode_character_sentiment.log --seconds-per-day 2 --bloom-intensity 0.3 -1440x900 --title "Sentiment for the Main Characters pr Episode" --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/graph_background.png`
    
    
- ` gource episode_character_sentiment.log --seconds-per-day 2 --bloom-intensity 0.3 -1440x900 --title "Sentiment for the Main Characters pr Episode" --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/graph_background.png --highlight-dirs -highlight-colour 808080`

# Episode, Sentiment pr Character (Main Characters pr. episode)

The idea about this gource is to:
- Episodes are branches to eachother (linked)
- For each episode, take the top 5 (total sentences) characters
- Color the character according to sentiment

In [291]:
#df_rel_sent

In [292]:
#df_episodes

In [304]:
temp_df = ps.sqldf(
    """
    
    select 
        episode
      , alias as character
      , compound as sentiment
      , rn
    from
    (
    select 
      df.*
    , row_number() over (partition by episode order by num_sentences desc) as rn
    from df_rel_sent as df
    )
    where lower(character) like '%rick%'
       or lower(character) like '%morty%'
       or lower(character) like '%summer%'
       or lower(character) like '%jerry%'
       or lower(character) like '%beth%'
    --where rn <= 5
    """
)
temp_df

,episode,character,sentiment,rn
0,Anatomy Park,Rick,0.8118,1
1,Anatomy Park,Morty,-0.5759,2
2,Anatomy Park,Jerry,0.9647,3
3,Anatomy Park,Beth,0.6852,7
4,Anatomy Park,Summer,-0.5000,9
...,...,...,...,...
120,Vindicators 3: The Return of Worldender,Rick,0.9959,1
121,Vindicators 3: The Return of Worldender,Morty,-0.9213,2
122,Vindicators 3: The Return of Worldender,Drunk Rick,-0.9381,4
123,Vindicators 3: The Return of Worldender,Beth,0.0000,7


- `1275543595|matthias|A|earthos/reef/1234/1`

In [305]:
temp_df.head(10)

,episode,character,sentiment,rn
0,Anatomy Park,Rick,0.8118,1
1,Anatomy Park,Morty,-0.5759,2
2,Anatomy Park,Jerry,0.9647,3
3,Anatomy Park,Beth,0.6852,7
4,Anatomy Park,Summer,-0.5000,9
5,Auto Erotic Assimilation,Rick,0.5279,1
6,Gotron Jerrysis Rickvangelion,Rick,0.9944,1
7,Gotron Jerrysis Rickvangelion,Summer,0.9942,2
8,Gotron Jerrysis Rickvangelion,Morty,0.6733,3
9,Gotron Jerrysis Rickvangelion,Beth,0.8852,4


In [306]:
def rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))
    r = int(max(0, 255*(ratio - 1)))
    g = 0#255 - b - r
    return r, g, b


def rgb_to_hex(rgb):
    return '%02x%02x%02x' % rgb

In [307]:
rgb_to_hex(rgb(1, -1, -1))

'ff0000'

In [308]:
# As we have 19 episodes
df_episode_unix_timestamp

,episode,timestamp,unixtime
0,1,2021-11-12 12:12:15.702384,1636719135
1,2,2021-11-13 12:12:15.702384,1636805535
2,3,2021-11-14 12:12:15.702384,1636891935
3,4,2021-11-15 12:12:15.702384,1636978335
4,5,2021-11-16 12:12:15.702384,1637064735
5,6,2021-11-17 12:12:15.702384,1637151135
6,7,2021-11-18 12:12:15.702384,1637237535
7,8,2021-11-19 12:12:15.702384,1637323935
8,9,2021-11-20 12:12:15.702384,1637410335
9,10,2021-11-21 12:12:15.702384,1637496735


In [309]:
res = pd.merge(
    temp_df,
    df_episodes[['name', 'id']],
    how='left',
    left_on='episode',
    right_on='name'
).sort_values(['id', 'rn'], ascending=True).reset_index(drop=True)
res

,episode,character,sentiment,rn,name,id
0,Pilot,Rick,0.9975,1,Pilot,1
1,Pilot,Morty,0.9971,2,Pilot,1
2,Pilot,Jerry,0.9619,3,Pilot,1
3,Pilot,Beth,0.8590,4,Pilot,1
4,Pilot,Summer,-0.7716,6,Pilot,1
...,...,...,...,...,...,...
120,Rickmurai Jack,Morty,0.9892,2,Rickmurai Jack,51
121,Rickmurai Jack,President Morty,0.8430,3,Rickmurai Jack,51
122,Rickmurai Jack,Jerry,0.3612,7,Rickmurai Jack,51
123,Rickmurai Jack,Beth,0.7646,8,Rickmurai Jack,51


In [311]:
res[res.episode =='Rickmurai Jack']

,episode,character,sentiment,rn,name,id
119,Rickmurai Jack,Rick,0.9806,1,Rickmurai Jack,51
120,Rickmurai Jack,Morty,0.9892,2,Rickmurai Jack,51
121,Rickmurai Jack,President Morty,0.8430,3,Rickmurai Jack,51
122,Rickmurai Jack,Jerry,0.3612,7,Rickmurai Jack,51
123,Rickmurai Jack,Beth,0.7646,8,Rickmurai Jack,51
124,Rickmurai Jack,Summer,0.6369,9,Rickmurai Jack,51


In [313]:
with open('episode_main_character_sentiment.log', 'w+') as f:
        

    n = 0
    for idx, row in res.iterrows():
        #print('1633360391' + '|' + str(row.episode))
        if idx == 0:
            episode = res.iloc[idx].episode
            episode_gathered = res.iloc[idx].episode
            folder = episode_gathered + '/'
            #print('idx0 :', episode)
            #print(res[res.episode == episode].character)
            #print(timestamp + '|' + 'sentiment|A|' + folder)
            f.write(timestamp + '|' + 'sentiment|A|' + folder + '\n')
            timestamp = str(df_episode_unix_timestamp.iloc[n].unixtime)
            for char in res[res.episode == episode].character:
                #print(episode + '/' + char + '.txt')
                #print(res[(res.episode == episode) & (res.character == char)].sentiment.values[0])
                sentiment_score = res[(res.episode == episode) & (res.character == char)].sentiment.values[0]
                sentiment_to_hex = rgb_to_hex(rgb(1, -1, sentiment_score))
                #print(sentiment_to_hex)
                #print(episode + '/' + char + '.txt' + '|' + sentiment_to_hex.upper())
                add_file = episode_gathered + '/' + char + '|' + sentiment_to_hex.upper()
                print(timestamp + '|' + 'sentiment|A|' + add_file)
                f.write(timestamp + '|' + 'sentiment|A|' + add_file + '\n')
                
        if idx != 0:
            episode = res.iloc[idx].episode 
            #for char in res[res.episode == episode].character:
            #    print(char)
            if episode != res.iloc[idx - 1].episode:
                episode = res.iloc[idx].episode
                if n == 0:
                    n += 1
                    continue
                try:
                    timestamp = str(df_episode_unix_timestamp.iloc[n].unixtime)
                except:
                    None
                #print('idx :', episode, res.iloc[idx].episode)
                episode_gathered += '/' + res.iloc[idx].episode
                if n % 3 == 0:
                    episode_gathered = res.iloc[idx].episode
                folder = episode_gathered + '/'
                print(timestamp + '|' + 'sentiment|A|' + folder)
                f.write(timestamp + '|' + 'sentiment|A|' + folder + '\n')
                #print('gathered : ', episode_gathered)
                for char in res[res.episode == episode].character:
                    sentiment_score = res[(res.episode == episode) & (res.character == char)].sentiment.values[0]
                    sentiment_to_hex = rgb_to_hex(rgb(1, -1, sentiment_score))
                    add_file = episode_gathered + '/' + char + '|' + sentiment_to_hex.upper()
                    print(timestamp + '|' + 'sentiment|A|' + add_file)
                    f.write(timestamp + '|' + 'sentiment|A|' + add_file + '\n')
                    
                n += 1
                #print(folder + '\n')
                #print('now : ', res.iloc[idx].episode)
                #print('before : ', res.iloc[idx-1].episode)
                #print(episode)
                #break
                

1636719135|sentiment|A|Pilot/Rick|0000FE
1636719135|sentiment|A|Pilot/Morty|0000FE
1636719135|sentiment|A|Pilot/Jerry|0000F5
1636719135|sentiment|A|Pilot/Beth|0000DB
1636719135|sentiment|A|Pilot/Summer|C40000
1636805535|sentiment|A|Pilot/Anatomy Park/
1636805535|sentiment|A|Pilot/Anatomy Park/Rick|0000CF
1636805535|sentiment|A|Pilot/Anatomy Park/Morty|920000
1636805535|sentiment|A|Pilot/Anatomy Park/Jerry|0000F5
1636805535|sentiment|A|Pilot/Anatomy Park/Beth|0000AE
1636805535|sentiment|A|Pilot/Anatomy Park/Summer|7F0000
1636891935|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/
1636891935|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/Rick|0000FE
1636891935|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/Morty|0000E8
1636891935|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/Jerry|0000FB
1636891935|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/Beth|0000B2
1636978335|sentiment|A|Meeseeks and Destroy/
1636978335|sentiment|A|Meeseeks and Destroy/Rick|0000

**Note:**
- Run with:
    - `gource episode_main_character_sentiment.log --seconds-per-day 2 --bloom-intensity 0.3 -1440x900 --title "Sentiment for the Main Characters pr Episode"`
    
    - `gource episode_main_character_sentiment.log --seconds-per-day 2 --bloom-intensity 0.3 --title "Sentiment for the Main Characters pr Episode" --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/graph_background.png --highlight-dirs -highlight-colour 000000 -o - | ffmpeg -y -r 60 -f image2pipe -vcodec ppm -i - -vcodec libx264 -preset ultrafast -crf 1 -threads 0 -bf 0 main_character_sentiment_episode.mp4`
    
    
- `gource episode_main_character_sentiment.log --seconds-per-day 2 --bloom-intensity 0.3 -1440x900 --title "Sentiment for the Main Characters pr Episode" --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/graph_background.png --highlight-dirs -highlight-colour 808080`

In [117]:
import math

def num_to_rgb(val, max_val=1):
    i = (val * 255 / max_val);
    r = round(math.sin(0.024 * i + 0) * 127 + 128);
    g = 0# round(math.sin(0.024 * i + 2) * 127 + 128);
    b = round(math.sin(0.024 * i + 4) * 127 + 128);
    return (r,g,b)

def rgb_to_hex(rgb):
    return '%02x%02x%02x' % rgb

print(rgb_to_hex(num_to_rgb(0)))

800020


In [30]:
res = ps.sqldf(
    """
    select 
        1633360391 || '|' || 'sentiment' || '|' || 'A' || '|' || episode as output
    from
    (
    
    select 
        episode
      , alias as character
      , compound as sentiment
      , rn
    from
    (
    select 
      df.*
    , row_number() over (partition by episode order by num_sentences desc) as rn
    from df_rel_sent as df
    )
    
    where rn <= 5
    )
    """
)

res

,output
0,1633360391|sentiment|A|Anatomy Park
1,1633360391|sentiment|A|Anatomy Park
2,1633360391|sentiment|A|Anatomy Park
3,1633360391|sentiment|A|Anatomy Park
4,1633360391|sentiment|A|Anatomy Park
...,...
106,1633360391|sentiment|A|Vindicators 3: The Retu...
107,1633360391|sentiment|A|Vindicators 3: The Retu...
108,1633360391|sentiment|A|Vindicators 3: The Retu...
109,1633360391|sentiment|A|Vindicators 3: The Retu...


In [31]:
with open('episode_character_sentiment.log', 'w') as f:
    for row in res.output:
        #print(row)
        f.write(row + '\n')

# Something Else

In [195]:
ps.sqldf(
    """
    select
          replace(df.name, ' ', '_') as character
        , df.episode
        , ts.timestamp
        , ts.unixtime
    from df_characters_exploded df
    inner join df_episode_unix_timestamp as ts
        on ts.episode = df.episode
    order by df.episode asc
    """
)

,character,episode,timestamp,unixtime
0,Rick_Sanchez,1,2021-10-04 15:13:11.890278,1633360391
1,Morty_Smith,1,2021-10-04 15:13:11.890278,1633360391
2,Bepisian,1,2021-10-04 15:13:11.890278,1633360391
3,Beth_Smith,1,2021-10-04 15:13:11.890278,1633360391
4,Canklanker_Thom,1,2021-10-04 15:13:11.890278,1633360391
...,...,...,...,...
1262,Redhead_Morty,51,2021-11-23 15:13:11.890278,1637680391
1263,Young_Jerry,51,2021-11-23 15:13:11.890278,1637680391
1264,Young_Beth,51,2021-11-23 15:13:11.890278,1637680391
1265,Young_Beth,51,2021-11-23 15:13:11.890278,1637680391


`unixtime || '|' || character || '|' || 'A' || '|' || 'episodes/episode_nr/' || episode || '.txt' as output`

In [214]:
# which characters touch which episode
res = ps.sqldf(
    """
    select 
        unixtime || '|' || character || '|' || 'A' || '|' || episode || '.txt' as output
    from
    
    (
    
    select 
       episode
    ,  character
    ,  unixtime
    from 
        (

        select
              replace(df.name, ' ', '_') as character
            , df.episode
            , ts.timestamp
            , ts.unixtime
        from df_characters_exploded df
        inner join df_episode_unix_timestamp as ts
            on ts.episode = df.episode
        order by df.episode asc
        )
    order by episode asc
    )
    """
)
res

,output
0,1633360391|Rick_Sanchez|A|1.txt
1,1633360391|Morty_Smith|A|1.txt
2,1633360391|Bepisian|A|1.txt
3,1633360391|Beth_Smith|A|1.txt
4,1633360391|Canklanker_Thom|A|1.txt
...,...
1262,1637680391|Redhead_Morty|A|51.txt
1263,1637680391|Young_Jerry|A|51.txt
1264,1637680391|Young_Beth|A|51.txt
1265,1637680391|Young_Beth|A|51.txt


In [215]:
#res = ps.sqldf(
#    """
#    
#    select 
#       unixtime || '|' || character || '|' || 'A' || '|' || 'episodes/episode_nr/' || episode || '.txt' as output
#    from 
#    (
#
#    select
#          replace(df.name, ' ', '_') as character
#        , df.episode
#        , ts.timestamp
#        , ts.unixtime
#    from df_characters_exploded df
#    inner join df_episode_unix_timestamp as ts
#        on ts.episode = df.episode
#    )
#    order by 1 desc
#    """
#)

In [219]:
with open('custom.log', 'w') as f:
    for row in res.output:
        #print(row)
        f.write(row + '\n')
        

- gource --hide dirnames,filenames --seconds-per-day 0.1 --auto-skip-seconds 1 -1280x720 -o - | ffmpeg -y -r 60 -f image2pipe -vcodec ppm -i - -vcodec libx264 -preset ultrafast -pix_fmt yuv420p -crf 1 -threads 0 -bf 0 gource.mp4

In [223]:
# which episodes that touch the characters
res = ps.sqldf(
    """
    select 
        unixtime || '|' || episode || '|' || 'A' || '|' || character || '.txt' as output
    from
    
    (
    
    select 
       episode
    ,  character
    ,  unixtime
    from 
        (

        select
              replace(df.name, ' ', '_') as character
            , df.episode
            , ts.timestamp
            , ts.unixtime
        from df_characters_exploded df
        inner join df_episode_unix_timestamp as ts
            on ts.episode = df.episode
        order by df.episode asc
        )
    order by episode asc
    )
    """
)
res

,output
0,1633360391|1|A|Rick_Sanchez.txt
1,1633360391|1|A|Morty_Smith.txt
2,1633360391|1|A|Bepisian.txt
3,1633360391|1|A|Beth_Smith.txt
4,1633360391|1|A|Canklanker_Thom.txt
...,...
1262,1637680391|51|A|Redhead_Morty.txt
1263,1637680391|51|A|Young_Jerry.txt
1264,1637680391|51|A|Young_Beth.txt
1265,1637680391|51|A|Young_Beth.txt


In [224]:
# something
res = ps.sqldf(
    """
    select 
        unixtime || '|' || episode || '|' || 'A' || '|' || character || '/' || episode || '.txt' as output
    from
    
    (
    
    select 
       episode
    ,  character
    ,  unixtime
    from 
        (

        select
              replace(df.name, ' ', '_') as character
            , df.episode
            , ts.timestamp
            , ts.unixtime
        from df_characters_exploded df
        inner join df_episode_unix_timestamp as ts
            on ts.episode = df.episode
        order by df.episode asc
        )
    order by episode asc
    )
    """
)
res

,output
0,1633360391|1|A|Rick_Sanchez/1.txt
1,1633360391|1|A|Morty_Smith/1.txt
2,1633360391|1|A|Bepisian/1.txt
3,1633360391|1|A|Beth_Smith/1.txt
4,1633360391|1|A|Canklanker_Thom/1.txt
...,...
1262,1637680391|51|A|Redhead_Morty/51.txt
1263,1637680391|51|A|Young_Jerry/51.txt
1264,1637680391|51|A|Young_Beth/51.txt
1265,1637680391|51|A|Young_Beth/51.txt
